In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import data as tfdata
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow import losses
from tensorflow.keras import initializers as init

from tensorflow.keras.utils import plot_model

DRL:  
    - 全称(Deep Reinforcement Learning) 深度强化学习  
    - 强调基于环境行动，以取得最大化预期利益  
    - alphaGO即是深度强化学习的典型应用  

这里使用**openAI** 提供的环境库 gym    
gym 主要由三个函数
- make
- render
- env

游戏中的两大引擎为物理引擎和图像引擎，gym中的render主要是负责图形的渲染，make则主要是物理引擎

这里我们使用深度强化学习玩cartPole(倒立摆)游戏。倒立摆是控制论中的经典问题，在这个游戏中，一根杆的底部通过轴与小车相连组成一个不稳定系统。  
我们需要通过控制小车左右移动，以使得杆一直保持垂直平衡状态

与gym的交互类似一个回合游戏:
- 首先获取游戏厨师状态
- 每个回合t中，选择一个动作交由gym执行
- gym 返回下一个状态和当前回合的收益值
- 持续迭代过程，直到游戏终止

这里使用深度强化学习中国的Deep Q-Learning方法来训练模型

## 定义超参

In [ ]:
import random
import  gym
# deque类似 list ,能够快速在两端添加或弹出
from collections import deque

#  训练总episodes
num_episodes = 500
# 探索过程所占的episode数量
num_exploration_episodes = 100
# 每个episode的最大回合数
max_len_episode = 1000
# 批次大小
batch_size = 32
# 学习率
learning_rate = 0.001
# 折扣因子
gamma = 1
# 探索起始时的探索率
initial_epsilon = 1.
# 探索终止时的探索率
final_epsilon = 0.01


## 定义模型
这里使用tf.kera.Model建立一个Q网络函数（Q-netword）,用于拟合Q learning中的Q函数   
这里我们使用较简单的全连接神经网络进行拟合，该网络输入当前状态，输出各个动作下的Q-value(CartPole下为2维)

In [ ]:
class QNetWork(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(units=24, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=24, activation=tf.nn.relu)
        self.dense3 = tf.keras.layers.Dense(units=2)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        return x
    
    def predict(self, inputs):
        q_values = self(inputs)
        return  tf.argmax(q_values, axis=-1)

## 模型训练

In [ ]:
# 实例化一个游戏环境，参数为游戏名称
env = gym.make("CartPole-v1")
# 实例化模型对象
model = QNetWork()
# 定义优化器
optimizer = optimizers.Adam(learning_rate=learning_rate)
# 使用deque作为Q Learning的经验回放池
replay_buffer = deque(maxlen=10000)

epsilon = initial_epsilon
# 开始探索迭代过程
for episode_id in range(num_episodes):
    # 初始化环境，获取初始状态
    state = env.reset()
    # 计算当前的探索率
    epsilon = max(
        initial_epsilon*(num_exploration_episodes - episode_id)/num_exploration_episodes,
        final_epsilon
    )

    for t in range(max_len_episode):
        # 对当前帧进行渲染
        env.render()
        # epsilon-greedy探索策略，以epsilon的概率选择随机动作
        if random.random() < epsilon:
            # 选择随机动作
            action = env.action_space.sample()
        else:
            # 模型计算出Q value最大的动作
            action = model.predict(np.expand_dims(state, axis=0)).numpy()
            action = action[0]
        
        # 让环境执行动作，或者执行玩动作的下一个状态，动作的奖励，游戏是否结束一级额外信息
        next_state, reward, done, info = env.step(action)
        # 如果游戏结束，则给予最大负奖励
        reward = -10. if done else reward
        # 将(state, action ,reward, next_state)放入经验回放池
        replay_buffer.append((state, action, reward, next_state, 1 if done else 0))
        # 更新state
        state = next_state

        # 游戏结束则退出本轮循环，进入下一个episode
        if done:
            print('episode %d \t epsilon %f \t score %d' %(episode_id, epsilon, t))
            break
        
        if len(replay_buffer) >= batch_size:
            # 从经验回放池中随机取一个批次的四元组，分别转化维Numpy数组
            batch_state, batch_action, batch_reward, batch_next_state, batch_done = zip(
                    *random.sample(replay_buffer, batch_size))
            batch_state, batch_reward, batch_next_state, batch_done = \
                    [np.array(a, dtype=np.float32) for a in [batch_state, batch_reward, batch_next_state, batch_done]]
            batch_action = np.array(batch_action, dtype=np.int32)

            q_value = model(batch_next_state)
            # reduce_max. 找出最大值
            # 计算y
            y = batch_reward + (gamma * tf.reduce_max(q_value, axis=1)) * (1 - batch_done)

            with tf.GradientTape() as tape:
                # 采用MSE来最小化距离
                loss = losses.mean_squared_error(
                    y_true=y,
                    y_pred=tf.reduce_sum(model(batch_state)*tf.one_hot(batch_action, depth=2), axis=1)
                )
            grads = tape.gradient(loss, model.variables)
            optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
